# < 위도, 경도 기준으로 인근 도시 매칭하기 >

* 작업 목적 : 당사가 조사한 세계의 몇몇 관광명소들이 당사의 DB에 없는 도시에 있어서 매칭이 안된 곳이다. 그래서 DB에 있는 가장 가까운 도시를 우선 연결짓기로 하였고 이 작업을 수행한 것이다


* 전달받은 데이터셋 : 당사 DB에 있는 도시 정보 , 도시 매칭이 되지 않은 관광명소의 좌표 정보

In [1]:
import pandas as pd
# 마이셀럽스 도시 데이터
city_data = pd.read_excel("/Users/myhellebs/Downloads/mbl_city_data_update.xlsx")
# 매칭이 필요한 관광명소 데이터
match_list = pd.read_excel("/Users/myhellebs/Downloads/도시 매칭 필요 리스트.xlsx")

In [2]:
# 데이터셋 확인
match_list.iloc[3].to_frame().T

,명소명,명소 주소,명소 위도,명소 경도,명소 타입
3,풀턴 체인 레이크스,"Fulton Chain Lakes , 미국 , 뉴욕",43.7499,-74.8163,호수


## * 당사의 기준에 따른 거리 공식을 차용하였음(피타고라스 정리 이용)

## (두 지점의 직선거리)^2 = (x축의 거리차)^2 + (y축의 거리차)^2 

이 공식을 이용해 관광명소와 당사 DB 속 도시와의 직선거리가 가장 가까운 곳을 매칭

In [3]:
import pandas as pd

def search_city(lat, lng, city_data):
    
    # 위도값이 +-5이내
    lat_list = city_data[(city_data["cd_latitude"] > lat - 5) & (city_data["cd_latitude"] < lat + 5)].copy()
    # 위도값이 +-5이내이면서 경도값이 +-5이내
    data = lat_list[(lat_list["cd_longitude"] > lng - 5) & (lat_list["cd_longitude"] < lng + 5)]
    # 두 지점의 직선거리를 계산한 칼럼
    data['cal'] = ((lat - data["cd_latitude"]) ** 2 + (lng - data["cd_longitude"]) ** 2) ** (1/2)
    data = data.sort_values(by = 'cal')
                    
    try :
        match_city = data.iloc[0]['cd_name']
    
    except :
        match_city = None
                    
    return match_city

In [5]:
pd.set_option('mode.chained_assignment',  None)
match_list["매칭 도시"] = match_list.apply(lambda x : search_city(x["명소 위도"], x["명소 경도"], city_data), axis = 1)


In [4]:
match_list.iloc[3].to_frame().T

,명소명,명소 주소,명소 위도,명소 경도,명소 타입
3,풀턴 체인 레이크스,"Fulton Chain Lakes , 미국 , 뉴욕",43.7499,-74.8163,호수


In [6]:
# 최종 데이터 셋 예시 (위와 비교)
match_list.iloc[3].to_frame().T

,명소명,명소 주소,명소 위도,명소 경도,명소 타입,매칭 도시
3,풀턴 체인 레이크스,"Fulton Chain Lakes , 미국 , 뉴욕",43.7499,-74.8163,호수,Old Forge


## - 이후에 해당 작업을 기반으로 여행 컨텐츠를 기획하는 마케팅팀 인턴들에게 인수인계 해주어 일의 효율을 높였음

In [7]:
import pandas as pd
city_data = pd.read_excel("/Users/myhellebs/Downloads/mbl_city_data_update.xlsx")
#city_data = city_data.drop('cd_idx',axis=1).drop('cd_city_id',axis=1)

lat,lng = map(float, input('위도와 경도를 \',\'표시로 구분지어 입력하세요 :').split(','))

data = city_data[(city_data["cd_latitude"] > lat-10) & (city_data["cd_latitude"] < lat+10)].copy().sort_values(by = "cd_latitude")
final = data[(data["cd_longitude"] > lng-10) & (data["cd_longitude"] < lng+10)].sort_values(by = 'cd_latitude')
final["cal"] = ((lat-final["cd_latitude"])**2 + (lng - final["cd_longitude"])**2)**(1/2)
final[["cd_name", "cd_name_ko","cd_country","cal"]].sort_values(by = "cal").head(3)

위도와 경도를 ','표시로 구분지어 입력하세요 :48.8606111,2.337644


,cd_name,cd_name_ko,cd_country,cal
5498,Paris,파리,fr,0.014378
5474,Saint-Ouen,생투앵,fr,0.051200
5550,Levallois-Perret,르발루아페레,fr,0.061025


In [ ]:
# 파리의 루브르 박물관 좌표를 입력하였더니 가장 가까운 도시 파리를 매칭 해줌 